In [1]:
from astropy import units as u
from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody import Orbit



In [2]:
r = [-6045, -3490, 2500] * u.km
v = [-3.457, 6.618, 2.533] * u.km / u.s

In [6]:
ss = Orbit.from_vectors(Earth, r, v)

In [7]:
ss

7283 x 10293 km x 153.2 deg (GCRS) orbit around Earth (♁) at epoch J2000.000 (TT)

In [8]:
ss.epoch

<Time object: scale='tt' format='jyear_str' value=J2000.000>

In [9]:
ss.epoch.iso

'2000-01-01 12:00:00.000'

In [10]:
ss.frame

<GCRS Frame (obstime=J2000.000, obsgeoloc=( 0.,  0.,  0.) m, obsgeovel=( 0.,  0.,  0.) m / s)>

In [11]:
ss.plot()

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(31, 119, 180)', 'dash':…

In [12]:
# Data for Mars at J2000 from JPL HORIZONS
a = 1.523679 * u.AU
ecc = 0.093315 * u.one
inc = 1.85 * u.deg
raan = 49.562 * u.deg
argp = 286.537 * u.deg
nu = 23.33 * u.deg

ss = Orbit.from_classical(Sun, a, ecc, inc, raan, argp, nu)

In [15]:
 ss.period.to(u.day)

<Quantity 686.9713834767824 d>

In [18]:
ss.v.to(u.mps)

AttributeError: module 'astropy.units' has no attribute 'mps'

In [19]:
u

<module 'astropy.units' from '/Users/Devansh/anaconda3/lib/python3.6/site-packages/astropy/units/__init__.py'>

In [22]:
from poliastro.examples import iss

In [23]:
iss

6772 x 6790 km x 51.6 deg (GCRS) orbit around Earth (♁) at epoch 2013-03-18 12:00:00.000 (UTC)

In [24]:
type(iss)

poliastro.twobody.orbit.Orbit

In [25]:
type(ss)

poliastro.twobody.orbit.Orbit

In [26]:
ss.plot()

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(31, 119, 180)', 'dash':…

In [27]:
from poliastro.maneuver import Maneuver

In [28]:
dv = [5, 0, 0] * u.m / u.s

man = Maneuver.impulse(dv)
man = Maneuver((0 * u.s, dv))  # Equivalent

In [29]:
 ss_i = Orbit.circular(Earth, alt=700 * u.km)

In [30]:
ss_i.plot()

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(31, 119, 180)', 'dash':…

In [31]:
hoh = Maneuver.hohmann(ss_i, 36000 * u.km)

In [33]:
hoh.get_total_cost()

<Quantity 3.6173999034657514 km / s>

In [42]:
hoh.get_total_time().to(u.hour)

<Quantity 4.369370318645549 h>

In [45]:
hoh.impulses[1]

(<Quantity 15729.733147123978 s>,
 <Quantity [    0.        ,-1420.00066617,    0.        ] m / s>)

In [46]:
ss_f = ss_i.apply_maneuver(hoh)

In [52]:
from poliastro.plotting import OrbitPlotter2D

In [53]:
op = OrbitPlotter2D()
ss_a, ss_f = ss_i.apply_maneuver(hoh, intermediate=True)
op.plot(ss_i, label="Initial orbit")
op.plot(ss_a, label="Transfer orbit")
op.plot(ss_f, label="Final orbit")

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(31, 119, 180)', 'dash':…

In [54]:
from astropy import time
epoch = time.Time("2015-05-09 10:43")  # UTC by default

In [55]:
epoch

<Time object: scale='utc' format='iso' value=2015-05-09 10:43:00.000>

In [56]:
from poliastro import ephem

/Users/Devansh/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88



In [57]:
Orbit.from_body_ephem(Earth, epoch)

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2015-05-09 10:44:07.185. Use Time(..., scale='tdb') instead.



1 x 1 AU x 23.4 deg (ICRS) orbit around Sun (☉) at epoch 2015-05-09 10:44:07.185 (TDB)

In [76]:
date_launch = time.Time('2011-11-26 15:02', scale='utc')
date_arrival = time.Time('2012-08-06 05:17', scale='utc')
tof = date_arrival - date_launch

ssei = Orbit.from_body_ephem(Earth, date_launch)
ssmi = Orbit.from_body_ephem(Mars, date_launch)
ssef = Orbit.from_body_ephem(Earth, date_arrival)
ssmf = Orbit.from_body_ephem(Mars, date_arrival)



from poliastro import iod
(v0, v), = iod.lambert(Sun.k, ssei.r, ssmf.r, tof)


r_e0, v_e0=ssei.rv()
ss_transfer = Orbit.from_vectors(Sun, r_e0, v0)



/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2011-11-26 15:03:06.183. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2012-08-06 05:18:07.183. Use Time(..., scale='tdb') instead.



In [77]:
v0

<Quantity [-29.29122325, 14.53256824,  5.41593207] km / s>

In [78]:
v

<Quantity [ 17.61580401,-10.99860504, -4.20763857] km / s>

In [79]:
op = OrbitPlotter2D()

op.plot(ssei, label="Initial Earth")
op.plot(ssef, label="Final Earth")
op.plot(ssmi, label="Initial Mars")
op.plot(ssmf, label="Final Mars")
op.plot(ss_transfer,label="Transfer")

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/propagation.py:232: UserWarning:

Frame <class 'astropy.coordinates.builtin_frames.icrs.ICRS'> does not support 'obstime', time values were not returned



FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(31, 119, 180)', 'dash':…

In [84]:
from poliastro.util import norm
norm(v0-v_e0)

<Quantity 3.318990996945012 km / s>